In [1]:
import pandas as pd

# ファイルがローカルにある場合（例としてCSV読み込みコードをコメントアウトで残します）
play_info_df = pd.read_csv('play_info.csv', encoding='cp932')

# 3分割する境界のインデックスを計算
total_rows = len(play_info_df)

# 1/3の境界点を計算
split_point_1 = total_rows // 3
# 2/3の境界点を計算
split_point_2 = 2 * (total_rows // 3)

# 3つのパートに分割
part1 = play_info_df.iloc[:split_point_1]
part2 = play_info_df.iloc[split_point_1:split_point_2]
part3 = play_info_df.iloc[split_point_2:] # 残りのすべて

# ファイルとして保存
part1.to_csv('play_info_part1.csv', index=False, encoding='cp932')
part2.to_csv('play_info_part2.csv', index=False, encoding='cp932')
part3.to_csv('play_info_part3.csv', index=False, encoding='cp932')

/var/folders/f_/zq3787px3l55ctv26lnhtydm0000gn/T/ipykernel_28422/716967477.py:4: DtypeWarning: Columns (125,141) have mixed types. Specify dtype option on import or set low_memory=False.
  play_info_df = pd.read_csv('play_info.csv', encoding='cp932')


In [2]:
import pandas as pd
import numpy as np

# --- データフレームの結合（元のコードから流用） ---
# df_part1, df_part2, df_part3 が事前に読み込まれていると仮定
# 例: df_part1 = pd.read_csv('part1.csv')
# ここではダミーデータを作成して動作をシミュレート
data = {
    'pitcher_name': ['A', 'B', 'C', None],
    'pitch_type_name': ['Fastball', 'Curve', None, 'Fastball'],
    'pre_ball': [1, 0, 2, 3],
    'pre_strike': [0, 1, 1, 2],
    'runner_2b_id': [np.nan, 'R2_123', np.nan, 'R2_456'],
    'runner_3b_id': ['R3_789', np.nan, np.nan, 'R3_101'],
    'other_col': [10, 20, 30, 40]
}
df_play = pd.DataFrame(data)

# ----------------------------------------------------

# 必要な列のリストを、データリストに合わせて修正
required_cols_for_dropna = [
    'pitcher_name', 
    'pitch_type_name', 
    'pre_ball',          # 修正: 'b_count' から変更
    'pre_strike',        # 修正: 's_count' から変更
    'runner_2b_id',      # 修正: 'is_runner_on_2b' から変更
    'runner_3b_id'       # 修正: 'is_runner_on_3b' から変更
]

# 1. 必要な列の欠損値処理
# runner_idを含む行でIDがNaNの場合、その行全体を削除
print("--- 処理前の行数:", len(df_play))
df_play.dropna(subset=required_cols_for_dropna, inplace=True)
print("--- 処理後の行数:", len(df_play))
print("\n--- 欠損値処理後のDataFrameのヘッダー ---")
print(df_play.head())

# 2. 型変換（ボールカウント/ストライクカウント）
# 欠損値処理により、これらの列はNaNを含まなくなったため、安全にintに変換可能
df_play = df_play.astype({
    'pre_ball': int,
    'pre_strike': int,
})

# 3. 走者情報のブール値（True/False）化
# runner_idが存在するかどうか（NaNではないか）をチェックし、
# 'is_runner_on_

--- 処理前の行数: 4
--- 処理後の行数: 0

--- 欠損値処理後のDataFrameのヘッダー ---
Empty DataFrame
Columns: [pitcher_name, pitch_type_name, pre_ball, pre_strike, runner_2b_id, runner_3b_id, other_col]
Index: []


In [3]:
#ピンチの時の配球
import pandas as pd
import numpy as np
import re

# ----------------------------------------------------------------------
# STEP 0: データの読み込み (cp932対応)
# ----------------------------------------------------------------------

print("--- STEP 0: データの読み込み (エンコーディング: cp932) ---")

files = [
    "play_info_part1.csv",
    "play_info_part2.csv",
    "play_info_part3.csv"
]

try:
    # cp932 (拡張 Shift JIS) で読み込み
    dfs = [pd.read_csv(f, encoding="cp932") for f in files]
    df = pd.concat(dfs, ignore_index=True)
    print("✅ 3つのCSVファイルを cp932 エンコーディングで読み込みました。")
    
except FileNotFoundError as e:
    print(f"エラー: {e}")
    print("❌ ファイルが見つからなかったため、分析処理をシミュレーションするためのダミーデータを作成します。")
    data = {
        'pitcher_name': ['戸郷 翔征', '戸郷 翔征', '村上 頌樹', '東 克樹', '東 克樹', '平良 海馬', '戸郷 翔征', '大野 雄大'],
        'pitch_type_name': ['Fastball', 'Splitter', 'Cutter', 'Fastball', 'Changeup', 'Slider', 'Splitter', 'Fastball'],
        'pre_ball': [1, 0, 2, 3, 0, 1, 1, 1],
        'pre_strike': [0, 1, 1, 2, 0, 0, 2, 0],
        'runner_1b_id': [np.nan, 'R1', 'R1', np.nan, np.nan, np.nan, np.nan, 'R1'],
        'runner_2b_id': [np.nan, 'R2', np.nan, 'R2', np.nan, 'R2', 'R2', np.nan],
        'runner_3b_id': ['R3', np.nan, np.nan, 'R3', np.nan, np.nan, np.nan, np.nan],
        'is_risp': [1, 0, 1, 1, 0, 1, 1, 0] # is_risp列をダミーに追加
    }
    df = pd.DataFrame(data)

# ----------------------------------------------------------------------
# STEP 1 & 2: データの正規化、絞り込み、ピンチ状況の定義
# ----------------------------------------------------------------------

# スペース除去関数（全角/半角すべて取る）
def normalize(name):
    if pd.isna(name):
        return ""
    return re.sub(r"\s+", "", str(name))

df["pitcher_norm"] = df["pitcher_name"].apply(normalize)

target_pitchers_raw = [
    "戸郷　翔征","村上　頌樹","颯","大野　雄大","栗林　良吏","石川　雅規", # 「颯」を「東 克樹」に修正して正規化リストを作成
    "伊藤　大海","鈴木　翔天","平良　海馬","横山　陸人","山岡　泰輔","大津　亮介"
]

# 正規化した比較用リスト
target_norm = [normalize(n) for n in target_pitchers_raw]

# 絞り込み
df_f = df[df["pitcher_norm"].isin(target_norm)].copy()

# ⚠️ is_risp列を使用する場合、元のデータにこの列が存在するか確認してください。
# 存在しない場合、以前のコードで定義した 'is_scoring_threat' を使用してください。
df_pinch = df_f[df_f["is_risp"] == 1].copy()

# ----------------------------------------------------------------------
# STEP 3: 分析の実行（ピンチ時の配球割合計算）
# ----------------------------------------------------------------------

print("\n--- STEP 3: 分析の実行と割合計算 ---")

# 全投手 × 全球種
all_pitch_types = sorted(df["pitch_type_name"].dropna().unique())
template = pd.MultiIndex.from_product(
    [target_norm, all_pitch_types],
    names=["pitcher_norm", "pitch_type_name"]
)

# 1. ピンチ時の投球数を集計し、テンプレートで全パターンを網羅（0埋め）
result = df_pinch.groupby(
    ["pitcher_norm", "pitch_type_name"]
).size().reindex(template, fill_value=0).reset_index(name="count")

# 2. transformを使用して割合(pct)を計算
result["pct"] = result.groupby("pitcher_norm")["count"].transform(
    lambda x: x / x.sum() * 100 if x.sum() > 0 else 0
)

result = result.sort_values(["pitcher_norm","pct"], ascending=[True,False])

# ----------------------------------------------------------------------
# STEP 4: 結果のCSVファイルへの書き出し (追加)
# ----------------------------------------------------------------------

output_filename = 'pitcher_pitching_analysis.csv'

try:
    # Shift JIS互換性のため cp932 で出力
    result.to_csv(output_filename, index=False, encoding='cp932')
    print(f"\n--- STEP 4: 結果の書き出し ---")
    print(f"✅ 分析結果を CSVファイル **'{output_filename}'** に保存しました。")
except Exception as e:
    print(f"❌ CSVファイルの書き出し中にエラーが発生しました: {e}")

# 結果の一部表示
print("\n--- 📝 結果サマリー (CSV保存データの一部) ---")
print(result.head(10))

--- STEP 0: データの読み込み (エンコーディング: cp932) ---


/var/folders/f_/zq3787px3l55ctv26lnhtydm0000gn/T/ipykernel_28422/2800655318.py:20: DtypeWarning: Columns (125) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = [pd.read_csv(f, encoding="cp932") for f in files]
/var/folders/f_/zq3787px3l55ctv26lnhtydm0000gn/T/ipykernel_28422/2800655318.py:20: DtypeWarning: Columns (125,141) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = [pd.read_csv(f, encoding="cp932") for f in files]
/var/folders/f_/zq3787px3l55ctv26lnhtydm0000gn/T/ipykernel_28422/2800655318.py:20: DtypeWarning: Columns (125,141) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = [pd.read_csv(f, encoding="cp932") for f in files]
/var/folders/f_/zq3787px3l55ctv26lnhtydm0000gn/T/ipykernel_28422/2800655318.py:20: DtypeWarning: Columns (125,141) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = [pd.read_csv(f, encoding="cp932") for f in files]
/var/folders/f_/

✅ 3つのCSVファイルを cp932 エンコーディングで読み込みました。

--- STEP 3: 分析の実行と割合計算 ---

--- STEP 4: 結果の書き出し ---
✅ 分析結果を CSVファイル **'pitcher_pitching_analysis.csv'** に保存しました。

--- 📝 結果サマリー (CSV保存データの一部) ---
    pitcher_norm pitch_type_name  count        pct
58          伊藤大海           ストレート    329  51.892744
61          伊藤大海            フォーク    130  20.504732
59          伊藤大海           スライダー    107  16.876972
56          伊藤大海            シュート     38   5.993691
54          伊藤大海          カットボール     21   3.312303
55          伊藤大海             カーブ      9   1.419558
57          伊藤大海            シンカー      0   0.000000
60          伊藤大海         チェンジアップ      0   0.000000
62          伊藤大海             特殊球      0   0.000000
106         大津亮介            フォーク     56  26.046512

--- STEP 3: 分析の実行と割合計算 ---

--- STEP 4: 結果の書き出し ---
✅ 分析結果を CSVファイル **'pitcher_pitching_analysis.csv'** に保存しました。

--- 📝 結果サマリー (CSV保存データの一部) ---
    pitcher_norm pitch_type_name  count        pct
58          伊藤大海           ストレート    329  51.892744
61     

In [4]:
#カウント別配球
import pandas as pd
import numpy as np
import re

# =========================================
# ■ 0. データの読み込み
# =========================================
print("--- STEP 0: データの読み込み ---")

files = [
    "play_info_part1.csv",
    "play_info_part2.csv",
    "play_info_part3.csv"
]

try:
    # cp932 (拡張 Shift JIS) で読み込み
    dfs=[pd.read_csv(f, encoding="cp932") for f in files]
    df=pd.concat(dfs, ignore_index=True)
    print("✅ 3つのCSVファイルを cp932 エンコーディングで読み込みました。")
    
except Exception as e:
    # 読み込み失敗時のエラー処理（ダミーデータ作成）
    print(f"❌ ファイル読み込みエラー: {e}")
    print("ダミーデータを作成して続行します。")
    data = {
        'pitcher_name': ['戸郷 翔征', '村上 頌樹', '東 克樹', '戸郷 翔征', '東 克樹', '平良 海馬', '大野 雄大'],
        'pitch_type_name': ['Fastball', 'Slider', 'Splitter', 'Cutter', 'Changeup', 'Slider', 'Fastball'],
        'pre_ball': [1, 0, 2, 3, 0, 1, 1],
        'pre_strike': [0, 1, 1, 2, 0, 0, 2],
        'runner_1b_id': [np.nan, 'R1', 'R1', np.nan, np.nan, 'R1', np.nan],
        'runner_2b_id': ['R2', np.nan, np.nan, 'R2', np.nan, 'R2', np.nan],
        'runner_3b_id': [np.nan, 'R3', np.nan, 'R3', np.nan, np.nan, np.nan]
    }
    df = pd.DataFrame(data)


# =========================================
# ■ 1. 前処理と列の準備
# =========================================
print("\n--- STEP 1: 前処理と列の準備 ---")

# スペース除去関数（全角/半角すべて取る）
def normalize(name):
    if pd.isna(name):
        return ""
    return re.sub(r"\s+", "", str(name))

# 投手名正規化列の作成
df["pitcher_norm"] = df["pitcher_name"].apply(normalize)

# 分析に必要なキー列の欠損値処理と型変換
required_cols = ["pitcher_name", "pre_ball", "pre_strike", "pitch_type_name"]
df.dropna(subset=required_cols, inplace=True)
df["pre_ball"] = df["pre_ball"].astype(int)
df["pre_strike"] = df["pre_strike"].astype(int)

# B-S 形式の 'count' 列を作成
df["count"] = df["pre_ball"].astype(str) + "-" + df["pre_strike"].astype(str)

# ランナー有無フラグの作成 (TypeError回避のための修正)
runner_cols = [c for c in df.columns if "runner" in c and "id" in c and "is" not in c]
runner_flags = df[runner_cols].notna().astype(int)
df["runner_exists"] = (runner_flags.sum(axis=1) > 0).astype(int)

# 分析対象の投手リスト（正規化前の全角スペース含む）
target_pitchers_raw = [
    "戸郷　翔征","村上　頌樹","颯","大野　雄大","栗林　良吏","石川　雅規", # 「颯」を「東 克樹」に修正して正規化リストを作成
    "伊藤　大海","鈴木　翔天","平良　海馬","横山　陸人","山岡　泰輔","大津　亮介"
]
target_norm = [normalize(n) for n in target_pitchers_raw]

# 12投手への絞り込み
df_f = df[df["pitcher_norm"].isin(target_norm)].copy()


# =========================================
# ■ 2. ① カウント別の配球傾向の分析
# =========================================
print("\n--- STEP 2: ① カウント別の配球傾向の分析 ---")
count_result = (
    df_f.groupby(["pitcher_norm", "count", "pitch_type_name"])
    .size()
    .reset_index(name="count_pitch")
)

# 割合を計算（transformを使用）
count_result["pct"] = (
    count_result["count_pitch"] /
    count_result.groupby(["pitcher_norm", "count"])["count_pitch"].transform("sum") * 100
)

count_result = count_result.sort_values(
    ["pitcher_norm", "count", "pct"],
    ascending=[True, True, False]
)


# =========================================
# ■ 3. ② ランナー有無の配球変化の分析
# =========================================
print("--- STEP 3: ② ランナー有無の配球変化の分析 ---")
runner_result = (
    df_f.groupby(["pitcher_norm", "runner_exists", "pitch_type_name"])
    .size()
    .reset_index(name="count_pitch")
)

# 割合を計算（transformを使用）
runner_result["pct"] = (
    runner_result["count_pitch"] /
    runner_result.groupby(["pitcher_norm", "runner_exists"])["count_pitch"].transform("sum") * 100
)

runner_result = runner_result.sort_values(
    ["pitcher_norm", "runner_exists", "pct"],
    ascending=[True, True, False]
)


# =========================================
# ■ 4. 結果の出力とCSV保存
# =========================================
print("\n--- STEP 4: 結果の出力とCSV保存 ---")
print("====== カウント別 配球傾向 (上位サンプル) ======")
print(count_result.head(30))

print("\n====== ランナー有無 配球変化 (上位サンプル) ======")
print(runner_result.head(30))

# CSV に保存 (cp932エンコーディング)
try:
    count_result.to_csv("count_pitch_distribute.csv", index=False, encoding="cp932")
    runner_result.to_csv("runner_pitch_distribute.csv", index=False, encoding="cp932")
    print("\n✅ 分析結果を 'count_pitch_distribute.csv' と 'runner_pitch_distribute.csv' に保存しました。")
except Exception as e:
    print(f"❌ CSVファイルの書き出し中にエラーが発生しました: {e}")


--- STEP 0: データの読み込み ---


/var/folders/f_/zq3787px3l55ctv26lnhtydm0000gn/T/ipykernel_28422/391544717.py:19: DtypeWarning: Columns (125) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs=[pd.read_csv(f, encoding="cp932") for f in files]
/var/folders/f_/zq3787px3l55ctv26lnhtydm0000gn/T/ipykernel_28422/391544717.py:19: DtypeWarning: Columns (125,141) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs=[pd.read_csv(f, encoding="cp932") for f in files]
/var/folders/f_/zq3787px3l55ctv26lnhtydm0000gn/T/ipykernel_28422/391544717.py:19: DtypeWarning: Columns (125,141) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs=[pd.read_csv(f, encoding="cp932") for f in files]
/var/folders/f_/zq3787px3l55ctv26lnhtydm0000gn/T/ipykernel_28422/391544717.py:19: DtypeWarning: Columns (125,141) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs=[pd.read_csv(f, encoding="cp932") for f in files]
/var/folders/f_/zq3787px3l55

✅ 3つのCSVファイルを cp932 エンコーディングで読み込みました。

--- STEP 1: 前処理と列の準備 ---

--- STEP 2: ① カウント別の配球傾向の分析 ---
--- STEP 3: ② ランナー有無の配球変化の分析 ---

--- STEP 4: 結果の出力とCSV保存 ---
====== カウント別 配球傾向 (上位サンプル) ======
   pitcher_norm count pitch_type_name  count_pitch        pct
3          伊藤大海   0-0           ストレート          263  33.040201
4          伊藤大海   0-0           スライダー          196  24.623116
6          伊藤大海   0-0            フォーク          147  18.467337
2          伊藤大海   0-0            シュート           64   8.040201
1          伊藤大海   0-0             カーブ           58   7.286432
0          伊藤大海   0-0          カットボール           52   6.532663
5          伊藤大海   0-0         チェンジアップ           15   1.884422
7          伊藤大海   0-0             特殊球            1   0.125628
11         伊藤大海   0-1           ストレート          160  37.914692
14         伊藤大海   0-1            フォーク           92  21.800948
12         伊藤大海   0-1           スライダー           91  21.563981
10         伊藤大海   0-1            シュート           38   9.004739
1

In [5]:
#ランナー有無別配球比較
import pandas as pd
import numpy as np

# CSVファイルを読み込む（cp932エンコーディングを使用）
df_runner = pd.read_csv("runner_pitch_distribute.csv", encoding="cp932")

# 1. 'runner_exists' の値を列名に含めるようにピボット
summary_pivot = df_runner.pivot_table(
    index=['pitcher_norm', 'pitch_type_name'],
    columns='runner_exists',
    values='pct',
    fill_value=0
)

# 2. 列名を分かりやすく変更
summary_pivot.columns = ['ランナーなし(%)', 'ランナーあり(%)']

# 3. 変化率（ピンチでどれだけ増減したか）を計算して追加
summary_pivot['変化率(%)'] = summary_pivot['ランナーあり(%)'] - summary_pivot['ランナーなし(%)']

# 4. 投手名でソートし、結果を見やすくする
summary_pivot = summary_pivot.sort_index(level=0)

# 5. ピボットテーブルをリセットし、選手名も列として表示する
summary_table = summary_pivot.reset_index()

# 表示のため、列の並び替えと四捨五入（小数点第1位まで）
summary_table['ランナーなし(%)'] = summary_table['ランナーなし(%)'].round(1)
summary_table['ランナーあり(%)'] = summary_table['ランナーあり(%)'].round(1)
summary_table['変化率(%)'] = summary_table['変化率(%)'].round(1)

# ランナーありの割合が高い順にソートして、頼り球を見やすくする
final_display = summary_table.sort_values(
    ['pitcher_norm', 'ランナーあり(%)'], 
    ascending=[True, False]
)

# ----------------------------------------------------
# 💡 ここからCSVファイル出力の処理を追加します
# ----------------------------------------------------

output_filename = 'pitcher_comparison_summary.csv'

try:
    # cp932エンコーディングでCSVファイルに書き出し
    final_display.to_csv(output_filename, index=False, encoding='cp932')
    print(f"\n--- 💾 結果のCSV保存 ---")
    print(f"✅ 全投手の配球サマリーを **'{output_filename}'** に保存しました。")
    print("このファイルをExcelなどで開くと、一覧で確認できます。")
    
except Exception as e:
    print(f"\n❌ CSVファイルの書き出し中にエラーが発生しました: {e}")

# 結果の表示（確認用）
print("\n--- ⚾ 全投手 ランナー有無別 配球割合サマリー (CSV保存データ) ---")
print(final_display)


--- 💾 結果のCSV保存 ---
✅ 全投手の配球サマリーを **'pitcher_comparison_summary.csv'** に保存しました。
このファイルをExcelなどで開くと、一覧で確認できます。

--- ⚾ 全投手 ランナー有無別 配球割合サマリー (CSV保存データ) ---
   pitcher_norm pitch_type_name  ランナーなし(%)  ランナーあり(%)  変化率(%)
3          伊藤大海           ストレート       40.6       47.9     7.4
6          伊藤大海            フォーク       18.3       22.2     3.8
4          伊藤大海           スライダー       23.7       17.6    -6.1
2          伊藤大海            シュート        4.9        6.8     1.9
0          伊藤大海          カットボール        5.1        3.1    -2.0
..          ...             ...        ...        ...     ...
59         鈴木翔天             カーブ        0.7        0.0    -0.7
64            颯            シュート       27.9       40.2    12.3
66            颯           スライダー       39.1       39.2     0.1
65            颯           ストレート       31.3       20.6   -10.7
63            颯             カーブ        1.7        0.0    -1.7

[67 rows x 5 columns]


In [38]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np
import re

# グラフ描画の共通設定
plt.rcParams['font.size'] = 12
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.family'] = 'Hiragino Sans'

# CSVファイルを読み込む（cp932エンコーディングを使用）
try:
    df_summary = pd.read_csv("pitcher_comparison_summary.csv", encoding="cp932")
except FileNotFoundError:
    print("エラー: 'pitcher_comparison_summary.csv' が見つかりません。")
    exit()

# 正規化関数（前セルと同じロジック）
def normalize(name):
    if pd.isna(name):
        return ""
    return re.sub(r"\s+", "", str(name))

# target_pitchers_raw:
target_pitchers_raw = [
    "戸郷　翔征", "村上　頌樹", "颯", "大野　雄大", "栗林　良吏", "石川　雅規",
    "伊藤　大海", "鈴木　翔天", "平良　海馬", "横山　陸人", "山岡　泰輔", "大津　亮介"
]

target_norm = [normalize(n) for n in target_pitchers_raw]

# グリッドの行/列数（3列 x 4行 = 12人）
n_cols = 3
n_rows = 4
# 図全体を大きく（dpiで拡大）
fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 8, n_rows * 6), dpi=350)

# タイトルをグリッド全体の一番上に大きく表示
fig.suptitle('全12投手の配球戦略の変化率\n(ランナーなしからピンチ時への変化)', fontsize=28, y=1.08)

# 各投手ごとにサブプロットで描画
for i, pitcher in enumerate(target_norm):
    row = i // n_cols
    col = i % n_cols
    ax = axes[row, col]

    df_pitcher_change = df_summary[df_summary['pitcher_norm'] == pitcher].copy()
    
    if df_pitcher_change.empty:
        ax.text(0.5, 0.5, f'{pitcher}\n(データなし)', ha='center', va='center', fontsize=16)
        ax.set_axis_off()
        continue

    # 表示用の前処理
    df_pitcher_change['abs_change'] = df_pitcher_change['変化率(%)'].abs()
    df_pitcher_change = df_pitcher_change[df_pitcher_change['abs_change'] >= 0.1]
    df_pitcher_change = df_pitcher_change.sort_values('abs_change', ascending=True)

    colors = ['#E44E57' if c > 0 else '#4C72B0' for c in df_pitcher_change['変化率(%)']]

    ax.barh(df_pitcher_change['pitch_type_name'], df_pitcher_change['変化率(%)'], color=colors)
    ax.set_title(pitcher, fontsize=18, fontweight='bold', color='#555555')
    ax.set_xlabel('変化率(%)', fontsize=15, labelpad=10)
    ax.set_ylabel('球種', fontsize=15, labelpad=10)
    ax.axvline(0, color='gray', linestyle='--')

    # 棒内のテキスト（色: 黒）
    for idx, val in enumerate(df_pitcher_change['変化率(%)']):
        x_pos = val * 0.5
        ha_align = 'right' if val > 0 else 'left'
        ax.text(x_pos, idx, f'{val:.1f}', va='center', ha=ha_align, fontsize=14, color='black', weight='bold')

# サブプロット間の余白をさらに狭く調整
plt.subplots_adjust(wspace=0.6, hspace=0.8, top=1.0)
plt.show()